In [22]:
import pandas as pd
import json
import emoji

# Function to decode emojis in text
def decode_emojis(text):
    if isinstance(text, str):
        return emoji.emojize(text, language='alias')
    return text

# Load the CSV data into a DataFrame
input_file_csv = 'instagram_messages.csv'  # Replace with your CSV file path
df = pd.read_csv(input_file_csv)

# Initialize a list to hold the JSON data
json_data = []


In [23]:

# Iterate through the DataFrame to create the prompt-response pairs
for i in range(len(df) - 1):
    current_row = df.iloc[i]
    next_row = df.iloc[i + 1]
    
    if current_row['sender_name'] == 'Divjot' or next_row['sender_name'] != 'Divjot':
        continue
    
    prompt_text = current_row['message']
    response_text = next_row['message']
    
    # Check if either the prompt or response is NaN
    if pd.isna(prompt_text) or pd.isna(response_text):
        continue
    
    json_data.append({
        "user": decode_emojis(prompt_text),
        "divjot": decode_emojis(response_text)
    })


In [24]:

# Convert the list to JSON format and save directly to output file
output_file = 'final_dataset.json'  # Replace with your desired output file path

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

print(f"Data has been saved to {output_file}")


Data has been saved to final_dataset.json
